In [16]:
import pandas as pd
import numpy as np
import pickle as pkl

In [81]:
# shuffle = True

results = pkl.load(open("../results/ist3/model_selection_shuffle_True.pkl", "rb"))

In [82]:
for learner, learner_results in results.items():
    print(learner)
    
    for sec in ["if_pehe", "pseudo_outcome_r", "pseudo_outcome_dr"]:
        
        print(learner, sec, np.mean(learner_results[sec]), np.std(learner_results[sec]))
    

XLearner
XLearner if_pehe 32.89394370530603 32.347509010739216
XLearner pseudo_outcome_r 0.4672908729027535 0.009089924086456171
XLearner pseudo_outcome_dr 0.9514744354360838 0.024823022214305067
DRLearner
DRLearner if_pehe 70.17459807804754 42.36233764836213
DRLearner pseudo_outcome_r 0.47478380637291534 0.00963699455888535
DRLearner pseudo_outcome_dr 0.9646376594435766 0.02362379470125624
SLearner
SLearner if_pehe 8.728692901697432 38.01495670809201
SLearner pseudo_outcome_r 0.46195778302558066 0.010367591626102773
SLearner pseudo_outcome_dr 0.9415154836912617 0.025107041167188558
TLearner
TLearner if_pehe 24.543818139182598 32.86135477688247
TLearner pseudo_outcome_r 0.46595873056852505 0.009841593138999615
TLearner pseudo_outcome_dr 0.9484421180686983 0.023292009965554063
RLearner
RLearner if_pehe 7.707658733529134 36.53364421822159
RLearner pseudo_outcome_r 0.46132932601635385 0.01101660623237128
RLearner pseudo_outcome_dr 0.9394581970210977 0.026094754509759437
RALearner
RALearne